<a href="https://colab.research.google.com/github/k23mhadh/DIABETES_ML_PROJECT/blob/main/diabetes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>